In [2]:
!pip3 install -q mediapipe-silicon

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks.python import vision
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import ast
import numpy as np
import os 
import pickle

objc[28167]: Class CaptureDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x10760c860) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2816f2548). One of the two will be used. Which one is undefined.
objc[28167]: Class CVWindow is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107210a68) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2816f2598). One of the two will be used. Which one is undefined.
objc[28167]: Class CVView is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107210a90) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/c

In [4]:
#open filtered data csv

data = pd.read_csv("new_data.csv")
data.head(11)

,class_name,video_id,split,bbox,link
0,fight,21704,test,"[75, 25, 218, 192]",https://www.signingsavvy.com/signs/mp4/22/2224...
1,fight,21706,train,"[81, 17, 232, 192]",https://www.signingsavvy.com/signs/mp4/22/2276...
2,find,69323,val,"[480, 69, 1350, 1080]",http://aslbricks.org/New/ASL-Videos/find.mp4
3,find,21858,train,"[79, 10, 213, 192]",https://www.signingsavvy.com/signs/mp4/6/6103.mp4
4,find,21859,train,"[68, 12, 210, 192]",https://www.signingsavvy.com/signs/mp4/9/9553.mp4
5,find,21863,train,"[187, 52, 572, 400]",http://www.aslsearch.com/signs/videos/pick.mp4
6,find,21852,train,"[30, 0, 276, 240]",https://media.spreadthesign.com/video/mp4/13/5...
7,find,21853,test,"[386, 53, 801, 720]",https://s3-us-west-1.amazonaws.com/files.start...
8,find,21854,train,"[682, 49, 1489, 1075]",https://signstock.blob.core.windows.net/signsc...
9,find,21855,test,"[112, 9, 555, 480]",https://media.asldeafined.com/vocabulary/14684...


In [13]:
column_names = ["Video_Id","Frame","Wrist","Thumb_CMC","Thumb_MCP","Thump_TIP","Index_Finger_MCP","Index_Finger_PIP","Index_Finger_DIP","Index_Finger_TIP","Middle_finger_MCP","Middle_finger_PIP","Middle_finger_DIP","Middle_finger_TIP","Ring_finger_MCP","Ring_finger_PIP","Ring_finger_DIP","Ring_finger_TIP","Pinky_finger_MCP","Pinky_finger_PIP","Pinky_finger_DIP","Pinky_finger_TIP"]
global info
info = pd.DataFrame()
video_id_list = list()
frame_list = []
wrist_list = []
thumb_cmc_list = []
thumb_mcp_list = []
thumb_tip_list = []
index_finger_mcp_list = []
index_finger_pip_list = []
index_finger_dip_list = []
index_finger_tip_list = []
middle_finger_mcp_list = []
middle_finger_pip_list = []
middle_finger_dip_list = []
middle_finger_tip_list = []
ring_finger_mcp_list = []
ring_finger_pip_list = []
ring_finger_dip_list = []
ring_finger_tip_list = []
pinky_finger_mcp_list = []
pinky_finger_pip_list = []
pinky_finger_dip_list = []
pinky_finger_tip_list = []

In [6]:
### WORKING ON INFO FOR MODEL

 
def model_info(link, index):
  
  mp_hands = mp.solutions.hands
  hands = mp_hands.Hands()
  mpDraw = mp.solutions.drawing_utils


  vid_path = link
  cap = cv2.VideoCapture(vid_path)
  frame_count = 0
  matching_rows = data[data['link'] == vid_path]
  
 
# Extract the 'video_id' from the matching row(s)
  video_id = matching_rows['video_id'][index]
  int(video_id)
  
  
  if not cap.isOpened():
    print("Error: Could not open video file.")
  
  else:
    while cap.isOpened():
  # Read a frame from the video
        ret, frame = cap.read()
        
  # Check if the frame was read successfully
        if not ret:
            break
 
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #recolor frame to run process

        detection = hands.process(frame_rgb)
  #If hands are detected then draw the hand landmarkd and the hand connections
        if detection.multi_hand_landmarks:
          frame_count +=1
          
          for landmarks in detection.multi_hand_landmarks:
            frame_list.append(frame_count)
            video_id_list.append(video_id)
            wrist_list.append([landmarks.landmark[0].x,landmarks.landmark[0].y])
            thumb_cmc_list.append([landmarks.landmark[1].x, landmarks.landmark[1].y])
            thumb_mcp_list.append([landmarks.landmark[2].x, landmarks.landmark[2].y])
            thumb_tip_list.append([landmarks.landmark[3].x, landmarks.landmark[3].y])
            index_finger_mcp_list.append([landmarks.landmark[5].x, landmarks.landmark[5].y])
            index_finger_pip_list.append([landmarks.landmark[6].x, landmarks.landmark[6].y])
            index_finger_dip_list.append([landmarks.landmark[7].x, landmarks.landmark[7].y])
            index_finger_tip_list.append([landmarks.landmark[8].x, landmarks.landmark[8].y])
            middle_finger_mcp_list.append([landmarks.landmark[9].x, landmarks.landmark[9].y])
            middle_finger_pip_list.append([landmarks.landmark[10].x, landmarks.landmark[10].y])
            middle_finger_dip_list.append([landmarks.landmark[11].x, landmarks.landmark[11].y])
            middle_finger_tip_list.append([landmarks.landmark[12].x, landmarks.landmark[12].y])
            ring_finger_mcp_list.append([landmarks.landmark[13].x, landmarks.landmark[13].y])
            ring_finger_pip_list.append([landmarks.landmark[14].x, landmarks.landmark[14].y])
            ring_finger_dip_list.append([landmarks.landmark[15].x, landmarks.landmark[15].y])
            ring_finger_tip_list.append([landmarks.landmark[16].x, landmarks.landmark[16].y])
            pinky_finger_mcp_list.append([landmarks.landmark[17].x, landmarks.landmark[17].y])
            pinky_finger_pip_list.append([landmarks.landmark[18].x, landmarks.landmark[18].y])
            pinky_finger_dip_list.append([landmarks.landmark[19].x, landmarks.landmark[19].y])
            pinky_finger_tip_list.append([landmarks.landmark[20].x, landmarks.landmark[20].y])
            data_dict = {
              'Frame': frame_list,
              'Video_Id': video_id_list,
              'Wrist': wrist_list,
              'Thumb_CMC': thumb_cmc_list,
              'Thumb_MCP': thumb_mcp_list,
              'Thumb_TIP': thumb_tip_list,
              'Index_Finger_MCP': index_finger_mcp_list,
              'Index_Finger_PIP': index_finger_pip_list,
              'Index_Finger_DIP': index_finger_dip_list,
              'Index_Finger_TIP': index_finger_tip_list,
              'Middle_Finger_MCP': middle_finger_mcp_list,
              'Middle_Finger_PIP': middle_finger_pip_list,
              'Middle_Finger_DIP': middle_finger_dip_list,
              'Middle_Finger_TIP': middle_finger_tip_list,
              'Ring_Finger_MCP': ring_finger_mcp_list,
              'Ring_Finger_PIP': ring_finger_pip_list,
              'Ring_Finger_DIP': ring_finger_dip_list,
              'Ring_Finger_TIP': ring_finger_tip_list,
              'Pinky_Finger_MCP': pinky_finger_mcp_list,
              'Pinky_Finger_PIP': pinky_finger_pip_list,
              'Pinky_Finger_DIP': pinky_finger_dip_list,
              'Pinky_Finger_TIP': pinky_finger_tip_list
          }
            

    new_data = pd.DataFrame(data_dict)
    return(new_data)

    data_dict.clear()
    cap.release()
 


In [14]:
#info.drop(info.index , inplace= True)

for i in range(len(data)):
    info = pd.concat([info, model_info(data['link'][i],i)])

print(info)

Empty DataFrame
Columns: []
Index: []


In [ ]:
#test run of the hand_land_marking function
#for i in range(10):
   # hand_land_marking(data['link'][i])
import concurrent.futures

def check_video_opening(link):
    try:
        vid = cv2.VideoCapture(link)
        opened = vid.isOpened()
        vid.release()
        return opened
    except Exception as e:
        print(f"Error opening video {link}: {e}")
        return False

not_open = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(check_video_opening, link) for link in data['link']]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        if not future.result():
            not_open.append(i)

print("Videos not opened:", not_open.length)

  